In [19]:
import os, json, yaml
from glob import glob
from collections import OrderedDict

In [9]:
records = {}

In [34]:
KEYS = ['process_name',
 'cross_section',
 'total_uncertainty',
 'DAS',
 'MCM',
 'equivalent_lumi',
 'accuracy',
 'fraction_negative_weight',
 'shower',
 'matrix_generator',
 'energy',
 'comments']

In [28]:
def key_order(key):
    priority = {
        "process_name": 1,
        "cross_section": 2,
        "total_uncertainty": 3,
        "DAS": 4,
        "MCM": 5,
        "equivalent_lumi": 6,
        "accuracy": 7,
        "fraction_negative_weight": 8,
        "shower": 9,
        "matrix_generator": 10,
        "energy": 11,
        "comments": 12
    }
    return priority.get(key, 999)

# 表示器类
class OrderedDumper(yaml.Dumper):
    pass

# 重写字典的表示方法
def _dict_representer(dumper, data):
    return dumper.represent_mapping(
        yaml.resolver.BaseResolver.DEFAULT_MAPPING_TAG,
        data.items(),
        flow_style=False
    )

OrderedDumper.add_representer(OrderedDict, _dict_representer)

# 递归排序函数
def sort_dict(d, key_order_func):
    if not isinstance(d, dict):
        return d
    
    sorted_dict = OrderedDict()
    for key in sorted(d.keys(), key=key_order_func):
        sorted_dict[key] = sort_dict(d[key], key_order_func)
    
    return sorted_dict

In [36]:
for f in glob('/afs/cern.ch/user/d/dfu/xsecdb/scripts/xsec_calc_auto/json_bak/QCD_Bin-PT*TuneCP5_13p6TeV*.json'):
    with open(f, 'r') as file:
        content = file.read()
    process_name = f.split('/')[-1].split('.')[0]
    data = json.loads(content)['records'][0]
    
    sorted_data = sort_dict(data, key_order)
    with open(f'{process_name}.txt', 'w') as f:
        f.write('\n'.join([str(data[k]) for k in KEYS]))